In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_216.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_98.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_137.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_86_1.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_19_1.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_202.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_159.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_217.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_185_1.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lifters/shop_lifter_n_40_1.mp4
/kaggle/input/shoplifting-videos-dataset/Shop DataSet/non shop lif

In [3]:
!nvidia-smi
!python -c "import torch; print(torch.version.cuda); print(torch.cuda.get_device_name(0)); print(torch.cuda.get_device_capability(0))"


Mon Nov 10 09:55:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install protobuf==3.20.3 --force-reinstall --no-cache-dir
!pip install transformers==4.44.2 accelerate==0.33.0 decord==0.6.0 torch torchvision --upgrade --no-cache-dir
!pip uninstall tensorflow -y
!pip install tensorflow-cpu==2.12.0 --no-cache-dir  # optional: avoids conflicts with HF or protobuf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-m

In [6]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_curve, roc_auc_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from decord import VideoReader, cpu
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification

2025-11-10 09:57:05.462652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762768625.684982      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762768625.749131      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
DATA_DIR = "/kaggle/input/shoplifting-videos-dataset/Shop DataSet"
OUTPUT_DIR = "/kaggle/working"
NUM_FRAMES = 16
FRAME_SIZE = 224
BATCH_SIZE = 2
EPOCHS = 3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
labels, corrupted = [], []

subdirs = [("shop lifters", 1), ("non shop lifters", 0)]
for folder, label in subdirs:
    folder_path = os.path.join(DATA_DIR, folder)
    if not os.path.exists(folder_path):
        continue
    for video_file in os.listdir(folder_path):
        if video_file.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
            video_path = os.path.join(folder_path, video_file)
            vid = cv2.VideoCapture(video_path)
            if not vid.isOpened():
                corrupted.append(video_path)
            else:
                labels.append([video_path, label])
            vid.release()

df = pd.DataFrame(labels, columns=["video_path", "label"])
df.to_csv(os.path.join(OUTPUT_DIR, "labels.csv"), index=False)
if corrupted:
    print(f"Corrupted videos: {len(corrupted)}")

In [ ]:
def check_empty_frames(video_path, threshold=10):
    vid = cv2.VideoCapture(video_path)
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    empty_frames = 0
    for i in range(min(100, frame_count)):
        ret, frame = vid.read()
        if ret and np.std(frame) < threshold:
            empty_frames += 1
    vid.release()
    return empty_frames / min(100, frame_count) > 0.5

df_filtered = []
for i, row in df.iterrows():
    if not check_empty_frames(row["video_path"]):
        df_filtered.append(row)
df = pd.DataFrame(df_filtered)
df.to_csv(os.path.join(OUTPUT_DIR, "labels_final.csv"), index=False)
print(f"Videos after removing empty frames: {len(df)}")

In [ ]:
min_count = min(df["label"].value_counts())
df_balanced = pd.concat([
    df[df["label"]==1].sample(min_count, random_state=42),
    df[df["label"]==0].sample(min_count, random_state=42)
])
df_balanced.to_csv(os.path.join(OUTPUT_DIR, "labels_balanced.csv"), index=False)
print(f"Balanced dataset size: {len(df_balanced)}")

In [ ]:
def check_video_properties(video_path, min_frames=NUM_FRAMES, min_resolution=(FRAME_SIZE, FRAME_SIZE)):
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        return False, None
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    vid.release()
    if frame_count < min_frames or width < min_resolution[0] or height < min_resolution[1]:
        return False, (frame_count, width, height)
    return True, (frame_count, width, height)

df_cleaned = []
for i, row in df.iterrows():
    valid, stats = check_video_properties(row["video_path"])
    if valid:
        df_cleaned.append(row)
df = pd.DataFrame(df_cleaned)
df.to_csv(os.path.join(OUTPUT_DIR, "labels_cleaned.csv"), index=False)
print(f"Videos after resolution/frame check: {len(df)}")

In [ ]:
def check_empty_frames(video_path, threshold=10):
    vid = cv2.VideoCapture(video_path)
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    empty_frames = 0
    for i in range(min(100, frame_count)):
        ret, frame = vid.read()
        if ret and np.std(frame) < threshold:
            empty_frames += 1
    vid.release()
    return empty_frames / min(100, frame_count) > 0.5

df_filtered = []
for i, row in df.iterrows():
    if not check_empty_frames(row["video_path"]):
        df_filtered.append(row)
df = pd.DataFrame(df_filtered)
df.to_csv(os.path.join(OUTPUT_DIR, "labels_final.csv"), index=False)
print(f"Videos after removing empty frames: {len(df)}")

In [ ]:
df = pd.read_csv("/kaggle/working/labels_balanced.csv")
print(f"Balanced dataset size: {len(df)}")
print(df['label'].value_counts())

In [ ]:
processor = VideoMAEImageProcessor.from_pretrained(MODEL_NAME)

class VideoMAEDataset(Dataset):
    def __init__(self, df, num_frames=16):
        self.df = df.reset_index(drop=True)
        self.num_frames = num_frames

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        video_path = self.df.iloc[idx]["video_path"]
        label = self.df.iloc[idx]["label"]

        vr = VideoReader(video_path, ctx=cpu(0))
        total_frames = len(vr)
        indices = np.linspace(0, total_frames - 1, self.num_frames, dtype=int)
        frames = vr.get_batch(indices).asnumpy()  # (T, H, W, C)

        inputs = processor(list(frames), return_tensors="pt")
        pixel_values = inputs["pixel_values"].squeeze(0)
        return pixel_values, torch.tensor(label)

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_dataset = VideoMAEDataset(train_df)
val_dataset = VideoMAEDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [ ]:
from transformers import VideoMAEForVideoClassification, VideoMAEImageProcessor

# Pretrained model from Hugging Face
model_name = "MCG-NJU/videomae-base"
processor = VideoMAEImageProcessor.from_pretrained(model_name)
model = VideoMAEForVideoClassification.from_pretrained(
    model_name,
    num_labels=2  # shoplifter / non-shoplifter
)
model = model.to(DEVICE)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    for pixel_values, labels in train_loader:
        pixel_values, labels = pixel_values.to(DEVICE), labels.to(DEVICE)
        outputs = model(pixel_values)
        loss = criterion(outputs.logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {running_loss/len(train_loader):.4f}")


In [ ]:
model.eval()
all_labels, all_probs = [], []

with torch.no_grad():
    for X, y in val_loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        outputs = model(pixel_values=X)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)[:, 1]
        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

all_labels = np.array(all_labels)
all_probs = np.array(all_probs)
all_preds = (all_probs >= 0.5).astype(int)

cm = confusion_matrix(all_labels, all_preds)
acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
print("Confusion Matrix:\n", cm)
print(f"Accuracy: {acc:.4f}, F1-score: {f1:.4f}")

In [ ]:
!nvidia-smi
!nvcc --version


In [ ]:
fpr, tpr, _ = roc_curve(all_labels, all_probs)
roc_auc = roc_auc_score(all_labels, all_probs)
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f"AUC = {roc_auc:.4f}")
plt.plot([0,1],[0,1],'--', color='gray')
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC Curve"); plt.legend(); plt.grid(True); plt.show()

precision, recall, _ = precision_recall_curve(all_labels, all_probs)
pr_auc = auc(recall, precision)
plt.figure(figsize=(6,6))
plt.plot(recall, precision, color='green', lw=2, label=f"PR AUC = {pr_auc:.4f}")
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title("Precision-Recall Curve"); plt.legend(); plt.grid(True); plt.show()